In [9]:
import os
import numpy as np 
import scipy.io.wavfile as wav 
import python_speech_features as psf
import hmmlearn.hmm as hmm

In [10]:
def extract_mfcc_features(filename):
    # load the wave file and get the sample rate and audio data
    sample_rate, audio = wav.read(filename)
    # extract mfcc features
    mfcc_features = psf.mfcc(audio, samplerate=sample_rate)
    return mfcc_features

In [11]:
extract_mfcc_features("./Gen/1/gen1.wav")

array([[-3.60436534e+01,  0.00000000e+00,  0.00000000e+00, ...,
         9.64028692e-15,  0.00000000e+00,  0.00000000e+00],
       [-3.60436534e+01,  0.00000000e+00,  0.00000000e+00, ...,
         9.64028692e-15,  0.00000000e+00,  0.00000000e+00],
       [-3.60436534e+01,  0.00000000e+00,  0.00000000e+00, ...,
         9.64028692e-15,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 8.74560559e+00, -1.22213688e+01, -1.03897968e+01, ...,
        -1.62279000e+01,  5.28178118e+00, -1.41650775e+01],
       [ 9.18190673e+00, -1.69296734e+01, -1.09801037e+01, ...,
        -2.31821082e+01,  1.76367379e+01, -1.67499842e+01],
       [ 8.83532433e+00, -1.38158269e+01, -6.39717537e+00, ...,
        -1.57318006e+01,  1.14734770e+01, -1.97949216e+01]])

In [16]:
# load the .wav file for each word in the training set
word_directories = ["../Day_2/Gen/1/","../Day_2/Gen/2/","../Day_2/Gen/3/",
                    "../Day_2/Gen/4/", "../Day_2/Gen/5/"] # directories where each of the word is contained
word_models = [] # initialize an empty list where we will store trained models

for word_directory in word_directories:
    word_mfcc_features = []
    for filename in os.listdir(word_directory):
        if filename.endswith(".wav"):
            filepath = os.path.join(word_directory, filename)
            word_mfcc_features.append(extract_mfcc_features(filepath))

    # stack mfcc features for all audio files in the directory
    word_mfcc_features = np.vstack(word_mfcc_features)
    
    # train an HMM model on the MFCC features
    num_states = 12 # hidden number of states.
    num_mixtures = 8 # btn 8 and 16

    # specify the starting probs
    start_prob = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

    word_model = hmm.GMMHMM(n_components=num_states, n_mix=num_mixtures, startprob_prior=start_prob)
    word_model.fit(word_mfcc_features)
    word_models.append(word_model)

/home/ix502iv/.local/lib/python3.10/site-packages/hmmlearn/utils.py:29: RuntimeWarning: invalid value encountered in divide
  a /= a_sum
/home/ix502iv/.local/lib/python3.10/site-packages/hmmlearn/hmm.py:788: RuntimeWarning: divide by zero encountered in divide
  self.covars_ = c_n / c_d
/home/ix502iv/.local/lib/python3.10/site-packages/hmmlearn/hmm.py:788: RuntimeWarning: invalid value encountered in divide
  self.covars_ = c_n / c_d
/home/ix502iv/.local/lib/python3.10/site-packages/hmmlearn/_emissions.py:196: RuntimeWarning: divide by zero encountered in log
  log_cur_weights = np.log(self.weights_[i_comp])
/home/ix502iv/miniconda3/envs/TensorFlow/lib/python3.10/site-packages/scipy/special/_logsumexp.py:110: RuntimeWarning: overflow encountered in exp
  tmp = np.exp(a - a_max)
/home/ix502iv/.local/lib/python3.10/site-packages/hmmlearn/hmm.py:788: RuntimeWarning: divide by zero encountered in divide
  self.covars_ = c_n / c_d
/home/ix502iv/.local/lib/python3.10/site-packages/hmmlearn/h

In [13]:
def validate_model(word_mfcc_features, word_models):
    log_likelihood = np.zeros(len(word_models))

    for i, model in enumerate(word_models):
        log_likelihood[i] = model.score(word_mfcc_features)

    # predict the word with highest likelihood
    word_index = np.argmax(log_likelihood)
    return word_index

In [14]:
word_index= validate_model(word_mfcc_features, word_models)

ValueError: startprob_ must sum to 1 (got nan)